In [1]:
import numpy as np
import numpy.linalg as la
import numpy.random as ra
import pprint

## Note

There are some codes where I did not use NumPy and other packages. I simply could have used them but I wanted to try to code the method manually using built-in functions in Python. I also created codes (without NumPy and other packages) for creating m by n zero matrices, creating n by n identity matrices, and performing matrix multiplication on an m by p to a p by n matrix. I could have used nested lists instead of NumPy arrays, but I have decided not to pursue it anymore since the assert statements require NumPy.

# <center> Motivation </center>

Consider the following problem,
\begin{equation}
\mathbf{A} x = c
\end{equation}
where $\mathbf{A}$ is an $N\times N$ matrix and $x$ and $c$ are column vectors. In particular, $x$ is a column vector with unknown components. The solution of this simple matrix equation is simply
\begin{equation}
x = \mathbf{A}^{-1}c.
\end{equation}
where we define
\begin{equation}
\mathbf{A} \cdot \mathbf{A}^{-1} = \mathbf{A}^{-1} \cdot \mathbf{A} = \mathbb{1}
\end{equation}
The elegance of this solution is that most of the computation is not dependent on $c$. Once we know $\mathbf{A}^{-1}$, then we can solve a general class of problems - we just need to plug in $c$!

Naively, the computational complexity of calculating the inverse is $N^3$. There are $N^2$ unknowns, corresponding to the $N\times N$ coefficients of $\mathbf{A}^{-1}$, and one would need to calculate the sum of $N$ products to isolate each uknown via Gaussian elimination.

After $\mathbf{A}^{-1}$, multiplying $\mathbf{A}^{-1}c$ involves $N^2$ operations.
___

However, there are times that computing the inverse of a matrix is unnecessary. We can skip directly to a computation that is $O(N^2)$. Suppose for example $\mathbf{A}$ is in an upper triangular form,
\begin{equation}
\mathbf{A} = \begin{pmatrix}
A_{11} & A_{12} & A_{13} & \dots & A_{1N} \\
0 & A_{22} & A_{23} &\dots & A_{2N} \\
0 & 0 & A_{33} & \ddots & \vdots \\
\vdots & \vdots & \ddots & \ddots & A_{N-1,N} \\
0 & 0 & \dots & 0 & A_{NN}
\end{pmatrix}
\end{equation}
The last row of $\mathbf{A} x = c$ yields an algebraic equation with a single unknown,
\begin{equation}
A_{NN} x_N = c_N \to x_N = \dfrac{A_{NN}}{c_N}. \left( \mathrm{should \; be} \dfrac{C_{N}}{A_{NN}} \right)
\end{equation}
With this value known, the second to the last row is also known
\begin{equation}
A_{N-1,N-1}x_{N-1} + A_{N-1,N} x_N = c_{N-1} \to x_{N-1} = \dfrac{1}{A_{N-1,N-1}} \left(c_{N-1} - A_{N-1,N} x_N\right)
\end{equation}
In fact, you should be able to convince yourself of the following recursion relation
\begin{equation}
x_i = \dfrac{1}{A_{i,i}} \left( c_i - \sum_{k = i+1}^N A_{i,k} x_k \right)
\end{equation}

___

Another example is a matrix in lower triangular form. Suppose $\mathbf{A}$ is of the form
\begin{equation}
\mathbf{A} = \begin{pmatrix}
A_{11} & 0 & 0 & \dots & 0 \\
A_{21} & A_{22} & 0 &\dots & 0 \\
A_{31} & A_{32} & A_{33} & \ddots & \vdots \\
\vdots & \vdots & \ddots & \ddots & 0 \\
A{N1} & A_{N2} & \dots & A_{N,N-1} & A_{NN}
\end{pmatrix}
\end{equation}
Following the same logic as before, you should be able to convience yourself that
\begin{equation}
x_i = \dfrac{1}{A_{i,i}} \left( c_i - \sum_{k=1}^{i-1} A_{i,k} x_k \right)
\end{equation}

## <center> Solving a linear set of equations involving lower and upper triangular matrices </center>

In the following, define two functions $\texttt{solveU}$ and $\texttt{solveL}$ which takes as input a upper and lower triangular matrix $\mathbf{U}$ and $\mathbf{L}$ respectively, and a column vector $c$, and respectively solves
\begin{equation}
\mathbf{U} x = c, \qquad \mathbf{L} x = c
\end{equation}

In [2]:
def zero_vector(n):
    """
    Creates an n-dimensional zero vector manually.
    
    where
        n: (int), number of elements.
    """
    result = []
    for i in range(0, n, 1):
        result.append(0)
    return result

In [3]:
def solveU(A, v):
    """
    Manually solves for x in the matrix equation A x = v.
    
    For n: (int), positive integer, where
        A: (list), an n by n non-singular upper-triangular coefficient matrix.
        v: (list), an n by 1 coefficient matrix.
        x: (list), an n by 1 unknown matrix.
    """
    x = zero_vector(len(v))
    x[-1] = v[-1] / A[-1, -1]
    for i in range(len(v) - 2, -1, -1):
        x[i] = (1 / A[i, i]) * (v[i] - sum(A[i, i + 1:] * x[i + 1:]))
    x = np.asarray(x)    #required in the next test
    return x

def solveL(A, v):
    """
    Manually solves for x in the matrix equation A x = v.
    
    For n: (int), positive integer, where
        A: (list), an n by n non-singular lower-triangular coefficient matrix.
        v: (list), an n by 1 coefficient matrix.
        x: (list), an n by 1 unknown matrix.
    """
    x = zero_vector(len(v))
    x[0] = v[0] / A[0, 0]
    for i in range(1, len(v), 1):
        x[i] = (1 / A[i, i]) * (v[i] - sum(A[i, 0:] * x[0:]))
    x = np.asarray(x)    #required in the next test
    return x

Your code should be able to run the following

In [4]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        U = np.triu(A)
        if np.log10(la.det(U)) > detbound:
            break
    
    c = ra.rand(N)
    xU = solveU(U,c)
    errorU = np.dot(U,xU) - c
    
    assert abs(np.dot(errorU, errorU)) < epsilon

In [5]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        L = np.tril(A)
        if np.log10(la.det(L)) > detbound:
            break
    
    c = ra.rand(N)
    xL = solveL(L,c)
    
    errorL = np.dot(L,xL) - c
    
    assert abs(np.dot(errorL, errorL))< epsilon

We want the two functions to assume that the inputted matrices are upper and lower triangular matrices, even though they may not be. This is useful later on, when the zeros may actually be floating points which are small - essentially coming from floating point arithmetic

In [6]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for i in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        U = np.triu(A)
        if np.log10(la.det(U)) > detbound:
            break
    
    c = ra.rand(N)
    xU1 = solveU(U,c)
    xU2 = solveU(A,c)
    errorU = xU1 - xU2
    
    assert abs(np.dot(errorU, errorU)) < epsilon

In [7]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for i in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        L = np.tril(A)
        if np.log10(la.det(L)) > detbound:
            break
    
    c = ra.rand(N)
    xL1 = solveL(L,c)
    xL2 = solveL(A,c)
    errorU = xL1-xL2
    
    assert abs(np.dot(errorL, errorL)) < epsilon

# <center> LU decomposition </center>

There's just one problem: almost all nonsingular matrices $\mathbf{A}$ are neither in upper triangular form and lower triangular form. What the LU decomposition lets us do is follow a deterministic process of factor $\mathbf{A}$ into a product of lower and upper triangular matrices $\mathbf{L}$ and $\mathbf{U}$ respectively.

That is, if we can find a factorization of $\mathbf{A}$ into $\mathbf{L} \mathbf{U}$, then the linear problem
\begin{equation}
\mathbf{A} x = c
\end{equation}
May be translated into
\begin{equation}
\mathbf{L} \mathbf{U} x = c
\end{equation}
Now if we define a new vector $v$ which satisfies
\begin{equation}
\mathbf{L} v = c
\end{equation}
The above equations reduces to two problems involving upper and lower triangular matrices.
\begin{equation}
\mathbf{U} x = v, \qquad \mathbf{L} v = c
\end{equation}
This we already know how to do!

___

The factorization of $\mathbf{A}$ into $\mathbf{L} \mathbf{U}$ proceeds by abusing the identity matrix and associativity. That is, we select a factorization of the identity matrix
\begin{equation}
\ell^{-1}_1 \ell_1 = \mathbb{1}
\end{equation}
such that applying the associative identity on 
\begin{equation}
\mathbb{1} \mathbf{A} = (\ell^{-1}_1 \ell_1) \mathbf{A} = \ell^{-1}_1 (\ell_1 \mathbf{A})
\end{equation}
makes the matrix $\ell_1 \mathbf{A}$ approach an upper triangular matrix and $\ell^{-1}_1$ approaches a lower triangular matrix. Then we do the process again
\begin{equation}
\ell^{-1}_1 \mathbb{1} (\ell_1 \mathbf{A}) = (\ell^{-1}_1 \ell^{-1}_2) (\ell_2 (\ell_1 \mathbf{A}))
\end{equation}
It's useful now to define a sequence of partially triangularized matrices $L^{(n)}$ and $U^{(n)}$, such that
\begin{equation}
L^{(n)} = L^{(n-1)} \ell^{-1}_n, \qquad U^{(n)} = \ell_n U^{(n-1)}
\end{equation}
and define
\begin{equation}
L^{(0)} = \mathbb{1}, \qquad U^{(0)} = A.
\end{equation}
Note the order of operations. Matrix products are associative, not commutative.

All that remains is to define the factorization of $\mathbb{1}$ such that this sequence of matrices terminates with an upper and lower triangular matrix.

## <center> Calculating $\ell_n$ and $U^{(n)}$ </center>

There is an infinite number of ways to create this sequence - most less optimal than others. Here is one: let $\ell_1$ be defined so that it has the most number of 0 possible (sparse to the max) and $U^{(1)}$ be a matrix with its first column looking like its from an upper triangular matrix. If you think hard enough, you might convince yourself that this is the optimal $\ell_1$,
\begin{equation}
\ell_1 = \mathbb{1} + [v^{(1)}, 0, 0, \dots 0]
\end{equation}
where
\begin{equation}
v^{(1)} = \left[0, -\dfrac{U^{(0)}_{2,1}}{U^{(0)}_{1,1}}, - \dfrac{U^{(0)}_{3,1}}{U^{(0)}_{1,1}}, \dots, \dfrac{U^{(0)}_{N,1}}{U^{(0)}_{1,1}} \right] ^T
\end{equation}
Just to be clear,
\begin{equation}
\ell_1 = \begin{pmatrix}
1 & 0 & 0 & \dots & 0 \\
-\dfrac{U^{(0)}_{2,1}}{U^{(0)}_{1,1}} & 1 & 0 & \dots & 0 \\
- \dfrac{U^{(0)}_{3,1}}{U^{(0)}_{1,1}} & 0 & 1 & \ddots & \vdots \\
\vdots & \vdots & \ddots & \ddots & 0 \\
\dfrac{U^{(0)}_{N,1}}{U^{(0)}_{1,1}} & 0 & \dots & 0 & 1
\end{pmatrix}
\end{equation}
Then
\begin{equation}
\ell_1 A = \ell_1 U^{(0)} = U^{(1)} = \begin{pmatrix}
U^{(1)}_{1,1} & U^{(1)}_{1,2} & U^{(1)}_{1,3} & \dots & U^{(1)}_{1,N} \\
0 & U^{(1)}_{2,2} & U^{(1)}_{2,3} & \dots & U^{(1)}_{2,N} \\
0 & U^{(1)}_{3,2} & U^{(1)}_{3,3} & \dots & U^{(1)}_{3,N} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & U^{(1)}_{N,2} & U^{(1)}_{N,3} & \dots & U^{(1)}_{N,N}
\end{pmatrix}
\end{equation}
Then, we do the same thing, defining a sparse-as-possible $\ell_2$ which forces the first two columns of $U^{(2)}$ to look like it comes from a upper triangular matrix.
\begin{equation}
\ell_2 = \mathbb{1} + [0, v_2, 0, \dots, 0]
\end{equation}
where
\begin{equation}
v^{(2)} = \left[0, 0, -\dfrac{U^{(1)}_{3,2}}{U^{(1)}_{2,2}}, -\dfrac{U^{(1)}_{4,2}}{U^{(1)}_{2,2}}, \dots, -\dfrac{U^{(1)}_{N,2}}{U^{(1)}_{2,2}} \right]^T
\end{equation}
___

In general,
\begin{equation}
\ell_n = \mathbb{1} + [0, 0, \dots, v^{(n)}, \dots, 0],
\end{equation}
where $v_n$ is located on the $n$th column and
\begin{equation}
v^{(n)}_k = \left\lbrace \begin{matrix}
0, & k \leq n \\
- \dfrac{U^{(n-1)}_{k,n}}{U^{(n-1)}_{n,n}}, & k > n
\end{matrix}\right.
\end{equation}
and
\begin{equation}
U^{(n)} = \ell_n U^{(n-1)}
\end{equation}

In the following code, let $\texttt{getvn}$ be a function whose inputs is a matrix and an order index $n$, which outputs the column vector above.

In [8]:
def getvn(A, n):
    """
    Intermediate result that solves for v(n).
    
    Where
        n: (int), 
        A: (array), an n by n non-singular coefficient matrix.
    """
    v = zero_vector(len(A))
    for k in range(0, len(A), 1):
        if k <= n - 1:
            v[k] == 0
        else:
            v[k] = - (A[k, n - 1] / A[n - 1, n - 1])
    v = np.asarray(v)    #required in the next test
    return v

The function should pass the following test

In [9]:
testmatrix = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

error = 1E-6
assert max(abs(getvn(testmatrix,1)-[0,-5,-9,-13])) < error
assert max(abs(getvn(testmatrix,2)-[0,0,-10/6,-14/6])) < error
assert max(abs(getvn(testmatrix,3)-[0,0,0,-15/11])) < error

The next function $\texttt{getelln}$ takes in an arbitrary matrix $U$ and an order index $n$, which return $\ell_n$. Useful functions here are $\texttt{np.identity(n)}$ which produces a $n\times n$ identity matrix.

In [10]:
def getelln(A, n):
    """
    Intermediate result that solves for l[n].
    
    Where
        n: (int), 
        A: (array), an n by n non-singular coefficient matrix.
    """
    v = np.append(
            np.append(
                np.zeros((len(A), n - 1)), 
                getvn(A, n).reshape((len(A), 1)),
                axis = 1
            ),
            np.zeros((len(A), len(A) - n)),
            axis = 1
        )
    l = v + np.identity(len(A))
    return l

The function should pass the following test

In [11]:
testmatrix = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

error = 1E-6

assert max(abs(getelln(testmatrix,1)[:,0]-[1,-5,-9,-13])) < error
assert max(abs(getelln(testmatrix,2)[:,1]-[0,1,-10/6,-14/6])) < error
assert max(abs(getelln(testmatrix,3)[:,2]-[0,0,1,-15/11])) < error

Now modify the previous code into a function $\texttt{updateU}$, whose input is a matrix $U$ and an order index $n$, which outputs the pair $v^{(n)}$ and $\ell_n U$

In [12]:
def updateU(A, n):
    """
    Solves for v(n) and solves for the product of l[n] and A.
    
    Where
        n: (int), 
        A: (array), an n by n non-singular coefficient matrix.
    """
    vn = getvn(A, n)
    Unew = np.matmul(getelln(A, n), A)
    return vn, Unew

The following code should show if $\texttt{updateU}$ results in an upper triangular matrix when applied thrice on a $4 \times 4$ matrix.

In [13]:
U0 = ra.rand(4,4)
print(U0)
v1, U1 = updateU(U0,1)
print(U1)
v2, U2 = updateU(U1,2)
print(U2)
v3, U3 = updateU(U2,3)
print(U3)

[[0.60448683 0.88600618 0.50872777 0.25024037]
 [0.30789435 0.5076997  0.5312376  0.69015455]
 [0.3795344  0.73551894 0.36457361 0.14269671]
 [0.2072832  0.7065147  0.29374949 0.73453663]]
[[ 6.04486834e-01  8.86006182e-01  5.08727771e-01  2.50240367e-01]
 [ 0.00000000e+00  5.64139512e-02  2.72117970e-01  5.62695041e-01]
 [-5.55111512e-17  1.79229198e-01  4.51627030e-02 -1.44197399e-02]
 [ 0.00000000e+00  4.02696336e-01  1.19302806e-01  6.48727276e-01]]
[[ 6.04486834e-01  8.86006182e-01  5.08727771e-01  2.50240367e-01]
 [ 0.00000000e+00  5.64139512e-02  2.72117970e-01  5.62695041e-01]
 [-5.55111512e-17  0.00000000e+00 -8.19366097e-01 -1.80212258e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.82314064e+00 -3.36792511e+00]]
[[ 6.04486834e-01  8.86006182e-01  5.08727771e-01  2.50240367e-01]
 [ 0.00000000e+00  5.64139512e-02  2.72117970e-01  5.62695041e-01]
 [-5.55111512e-17  0.00000000e+00 -8.19366097e-01 -1.80212258e+00]
 [ 1.23515771e-16  0.00000000e+00  2.22044605e-16  6.41909965e-01]]


## <center> Calculating $L^{(n)}$ </center>

One thing we've left out is how $L^{(n)}$ is calculated. Let us recall the recursion,
\begin{equation}
L^{(n)} = L^{(n-1)} \ell^{-1}_n.
\end{equation}
What is nice about the our definition of $\ell_n$ is that
\begin{equation}
\ell^{-1}_n = \mathbb{1} + [0, 0, \dots, -v^{(n)}, \dots, 0]
\end{equation}
Even more convenient (and you should check this) is that
\begin{equation}
L^{(n)} = \mathbb{1} + [-v^{(1)}, -v^{(2)}, \dots, - v^{(n-1)}, - v^{(n)}, 0 \dots, 0]
\end{equation}
In fact our final $L = L^{(N-1)}$ is simply
\begin{equation}
L = \mathbb{1} + [-v^{(1)}, -v^{(2)}, \dots, - v^{(N-2)}, - v^{(N-1)}, 0]
\end{equation}

## <center> Solving linear sets of equations </center>

Now it's finally time to combine everything we know. First, let us define a function $\texttt{getLU}$, whose input is an arbitrary function $\mathbf{A}$ and whose output are two matrices $\mathbf{L}$ and $\mathbf{U}$ which are the LU factorization of $\mathbf{A}$.

Note that for an $N\times N$ matrix, one needs to use $\texttt{updateU}$ $N-1$ times. Also, one can use the vectors outputted by $\texttt{updateU}$ to construct $\mathbf{L}$.

A useful function here is $\texttt{np.copy}$, which lets you copy the values of a matrix, so that mutations on a copied matrix does not affect the original matrix.

In [14]:
def getLU(A):
    """
    Solves the LU decomposition of A.
    
    Where
        A: (array), an n by n non-singular coefficient matrix.
    """
    U = A.copy()
    L = np.empty((len(A), 0))
    for j in range(0, len(A), 1):
        v, U = updateU(U, j + 1)
        L = np.append(L, (- v).reshape((len(A), 1)), axis = 1)
    L += np.identity(len(A))
    return L, U

If the factorization is correct, then $\mathbf{L} \mathbf{U} = A$

In [15]:
N = 20
A = ra.rand(N,N)
L,U = getLU(A)

assert la.norm(np.dot(L,U) - A) < 1E-6

Now let's combine everything finally. Given a matrix $\mathbf{A}$ and a constant column vector $c$, we solve for an unknown set of coefficients $x$ constrained by the following set of linear algebraic equations:
\begin{equation}
\mathbf{A} x = c
\end{equation}
We first factorize $\mathbf{A}$ into $\mathbf{L} \mathbf{U}$ and first solve a intermediate set of coefficients $v$
\begin{equation}
\mathbf{L}v = c
\end{equation}
using $\texttt{solveL}$ and then finally $x$
\begin{equation}
\mathbf{U}x = v
\end{equation}
using $\texttt{solveU}$.

Kindly implement this in the following function $\texttt{solveAxc}$, whose inputs are $\mathbf{A}$ and $c$ and whose output is the solution $x$

In [16]:
def solveAxc(A,c):
    """
    Solves for x in the matrix equation A x = c using LU decomposition.
    
    For n: (int), positive integer, where
        A: (list), an n by n non-singular coefficient matrix.
        c: (list), an n by 1 coefficient matrix.
        x: (list), an n by 1 unknown matrix.
    """
    L, U = getLU(A)
    v = solveL(L, c)
    x = solveU(U, v)
    return x

Your solution should pass the following unit test.

In [17]:
N = 20
A = ra.rand(N,N)
c = ra.rand(N)
x = solveAxc(A,c)

assert la.norm(np.dot(A,x) - c)<1E-12